In [ ]:
from sqlalchemy import create_engine ,Integer, String, Date,VARCHAR,text
import pandas as pd

In [ ]:

pd.set_option('display.max_columns', None)
caminho_do_arquivo = r"C:\Users\edmil\OneDrive\Documentos\Cursos\Python - SGBDS - Arquivos\Origem de dados\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')
colunas = ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrência", "Data_da_Ocorrencia","Municipio","UF","Regiao","Nome_do_Fabricante","Modelo"]
df = df[colunas]
df.rename( columns={  'Classificacao_da_Ocorrência' : 'Classificacao_da_Ocorrencia'  } ,inplace=True )


In [ ]:
df.head()

In [ ]:
df.shape[0]

In [ ]:
from datetime import datetime, timedelta

#Método 1
data_atual = datetime.now()# Data Atual
data_ha_3_meses = data_atual - timedelta(days=3*30) # Subtrai 3 meses usando timedelta 
data_ha_3_meses_apenas_data = data_ha_3_meses.date() # traz somente a data , exclui hora minuto segundo etc

#Obs:Não leva em consideração a variação nos números de dias em cada mês, resultando em uma aproximação.
print(data_ha_3_meses_apenas_data)

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

#Método 2
data_atual2 = datetime.now()
d_menos3M = data_atual - relativedelta(months=3) # Subtrai 3 meses usando relativedelta
dt_d_menos3M = d_menos3M.date()

#Obs:Leva em consideração a variação nos números de dias em cada mês, fornecendo um resultado mais preciso
print(dt_d_menos3M)


In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
# Converter tudo para Pandas to_datetime para fazer o filtro na base 
df['Data_da_Ocorrencia']    = pd.to_datetime(df['Data_da_Ocorrencia'])
data_ha_3_meses_apenas_data = pd.to_datetime(data_ha_3_meses_apenas_data)
dt_d_menos3M                = pd.to_datetime(dt_d_menos3M)

In [ ]:
df.shape[0]

In [ ]:
df = df[df['Data_da_Ocorrencia'] >= dt_d_menos3M]

In [ ]:
df.head()

In [ ]:
dt_d_menos3M_ajustado=dt_d_menos3M.date()

In [ ]:
#conexao com banco de dados
dbname   = 'python'
user     = 'postgres'
password = '12345'
host     = 'localhost'
port     = '5432' 

conexao_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(conexao_str)

nome_tabela = 'anac_sqlalchemy' 

# Deletar registros com base no ano atual
cursor=engine.connect() 
delete = text(f'''    delete from public.{nome_tabela}	
                      WHERE "Data_da_Ocorrencia" >=  '{dt_d_menos3M_ajustado}'
                ''')
cursor.execute(delete)
cursor.commit()

# Enviar DataFrame para o banco de dados
df.to_sql(nome_tabela, engine, index=False, if_exists='append',
                     dtype={ 
                           'Numero_da_Ocorrencia' :   Integer ,
                           'Classificacao_da_Ocorrencia': VARCHAR(50),
                           'Data_da_Ocorrencia':Date  
                           })

engine.dispose()
cursor.close()